# Summary of this notebook

...

In [1]:
#If using Google Colab and Google Drive, run the following commands

#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#REPLACE THIS COMMAND WITH THE APPROPRIATE PATH TO THE "code" FOLDER ON YOUR GOOGLE DRIVE
#%cd ./drive/MyDrive/poverty_project/group_project/code

### Imports

In [3]:
import pandas as pd
import numpy as np

#from os import listdir
#from os.path import isfile, join

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

### Load data

In [4]:
train = pd.read_csv('../data/train.csv', index_col='index')
test = pd.read_csv('../data/test.csv', index_col='index')
train.head()

,country,year,wealth_index,is_urban,households,latitude,longitude,is_poorest
index,,,,,,,,
lesotho_2014_00-254,lesotho,2014.0,0.983558,1.0,24.0,-28.909042,27.936529,0
burkina_faso_2010_00-162,burkina_faso,2010.0,-0.604069,0.0,25.0,11.163129,-3.322228,1
mali_2012_00-381,mali,2012.0,-0.371823,0.0,25.0,11.877264,-6.966146,0
lesotho_2009_00-237,lesotho,2009.0,-0.110702,0.0,23.0,-29.518770,28.033241,0
zimbabwe_2010_00-125,zimbabwe,2010.0,-0.749686,0.0,28.0,-16.217611,30.610455,1


In [5]:
test.head()

,country,year,wealth_index,is_urban,households,latitude,longitude,is_poorest
index,,,,,,,,
kenya_2014_17-28,kenya,2014.0,-0.199972,1.0,20.0,-0.907212,34.582047,0
kenya_2014_31-11,kenya,2014.0,-0.311057,0.0,23.0,0.541424,34.334202,0
zimbabwe_2015_00-334,zimbabwe,2015.0,1.621408,1.0,28.0,-18.985624,32.593941,0
lesotho_2009_00-370,lesotho,2009.0,-0.373018,0.0,21.0,-29.093271,27.666286,0
zambia_2013_00-466,zambia,2013.0,-0.556459,0.0,23.0,-17.476536,26.537828,1


In [6]:
train['is_poorest'].value_counts(normalize=True)

0    0.665411
1    0.334589
Name: is_poorest, dtype: float64

In [7]:
test['is_poorest'].value_counts(normalize=True)

0    0.663244
1    0.336756
Name: is_poorest, dtype: float64

### Create dummies

In [8]:
train = pd.get_dummies(train, columns=['country'], drop_first=True)
len(train.columns)

29

In [9]:
test = pd.get_dummies(test, columns=['country'], drop_first=True)
len(test.columns)

29

### Create funtion to run models

In [10]:
def fit_and_report(model, df_train, df_test, is_print=False):
    # Split X, y
    only_country = ['year', 'wealth_index', 'households', 'is_poorest', 'latitude', 'longitude', 'is_urban']
    country_is_urban = ['year', 'wealth_index', 'households', 'is_poorest', 'latitude', 'longitude']
    country_is_urban_lat_long = ['year', 'wealth_index', 'households', 'is_poorest']
    y = 'is_poorest'
    
    df_train_f1 = df_train.drop(columns=only_country)
    df_train_f2 = df_train.drop(columns=country_is_urban)
    df_train_f3 = df_train.drop(columns=country_is_urban_lat_long)
    X_train = [df_train_f1, df_train_f2, df_train_f3]
    y_train = df_train[y]
    
    df_test_f1 = df_test.drop(columns=only_country)
    df_test_f2 = df_test.drop(columns=country_is_urban)
    df_test_f3 = df_test.drop(columns=country_is_urban_lat_long)
    X_test = [df_test_f1, df_test_f2, df_test_f3]
    y_test = df_test[y]
    
    scores = []
    for i in range(3):
        model[i].fit(X_train[i], y_train)
        scores.append(model[i].score(X_train[i], y_train))
        scores.append(model[i].score(X_test[i], y_test))
        if is_print:
            print(f'feature {i+1} train = {model[i].score(X_train[i], y_train)}')
            print(f'feature {i+1} test = {model[i].score(X_test[i], y_test)}\n')
    return scores

### Modeling & Result

In [11]:
# Prepare datafram to keep the results
df_score = pd.DataFrame(index=['only_country_train', 'only_country_test', 
                               'country_is_urban_train', 'country_is_urban_test', 
                               'country_is_urban_lat_long_train', 'country_is_urban_lat_long_test'])

# Iterate through all models for making the baseline model
logr, rf, ada, knn = [], [], [], []
for _ in range(3):
    logr.append(LogisticRegression(random_state=42))
    rf.append(RandomForestClassifier(random_state=42))
    ada.append(AdaBoostClassifier(random_state=42))
    knn.append(KNeighborsClassifier())
df_score['LogisticRegression'] = fit_and_report(logr, train, test);
df_score['RandomForestClassifier'] = fit_and_report(rf, train, test);
df_score['AdaBoostClassifier'] = fit_and_report(ada, train, test);
df_score['KNeighborsClassifier'] = fit_and_report(knn, train, test);

df_score

/Users/atigon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/atigon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to 

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,KNeighborsClassifier
only_country_train,0.709247,0.709075,0.709247,0.628596
only_country_test,0.709446,0.704312,0.709446,0.605749
country_is_urban_train,0.799886,0.799886,0.799886,0.782763
country_is_urban_test,0.783368,0.783368,0.783368,0.764887
country_is_urban_lat_long_train,0.790354,0.999829,0.817637,0.880936
country_is_urban_lat_long_test,0.781314,0.820329,0.808008,0.813142


### Export

In [13]:
df_score.to_csv('../data/baseline_model_score.csv')